In [6]:
import numpy as np
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta, date
import matplotlib.pylab as plt
from pandas.plotting import register_matplotlib_converters
%matplotlib inline
import re
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [7]:
mostRecentDay = date(2025, 1, 1)
startingDay = date(mostRecentDay.year-10, mostRecentDay.month, mostRecentDay.day)

In [8]:
allStockTickers = pd.read_excel("23-24 Competition Stock List-FINAL.xlsx")
allStockTickers = allStockTickers[["Ticker", "GICS Sector", "Exchange"]]
allStockTickers

,Ticker,GICS Sector,Exchange
0,ABEV3,Consumer Staples,B3 S.A.
1,CRFB3,Consumer Staples,B3 S.A.
2,B3SA3,Financials,B3 S.A.
3,BPAC11,Financials,B3 S.A.
4,BBAS3,Financials,B3 S.A.
...,...,...,...
520,OTEX,Information Technology,Toronto Stock Exchange
521,QSR,Consumer Discretionary,Toronto Stock Exchange
522,RY,Financials,Toronto Stock Exchange
523,SHOP,Information Technology,Toronto Stock Exchange


In [24]:
allCSVFiles = ["DAAA"]
allFeatures = []

def createTable(filename):
    df = pd.read_csv(filename+".csv")
    df["DATE"] = pd.to_datetime(df['DATE']).ffill()
    for idx, val in enumerate(df[filename]):
        try:
            float(val)
        except:
            df[filename].iloc[idx] = None
    df[filename] = df[filename].astype(float).fillna(0.0)
    df = df.set_index("DATE")
    monthlydf = df.resample('D').mean()
    monthlydf["DATE"] = monthlydf.index.date
    monthlydf = monthlydf.reset_index(drop=True)
    valid = False
    for date in monthlydf["DATE"]:
        if date >= mostRecentDay:
            valid = True
    if not valid:
        print("ERROR")
    monthlydf.drop(monthlydf[monthlydf.DATE < startingDay].index, inplace=True)
    monthlydf.drop(monthlydf[monthlydf.DATE > mostRecentDay].index, inplace=True)
    monthlydf['DATE'] = pd.to_datetime(monthlydf['DATE']).dt.to_period('D').dt.to_timestamp()
    monthlydf
    monthlydf.set_index('DATE', inplace=True)
    return monthlydf

for file in allCSVFiles:
    allFeatures.append(createTable(file))
print(allFeatures)

[            DAAA
DATE            
2015-01-01  0.00
2015-01-02  3.67
2015-01-03   NaN
2015-01-04   NaN
2015-01-05  3.59
...          ...
2024-12-28   NaN
2024-12-29   NaN
2024-12-30  5.35
2024-12-31  5.40
2025-01-01  0.00

[3654 rows x 1 columns]]


In [9]:
def gatherStockPrices(ticker = str):
    pd.options.mode.chained_assignment = None
    end = mostRecentDay
    start = datetime(end.year-10, end.month, end.day)
    df = yf.download(ticker, start=start, end=end, interval = "1d")
    df = df[["Close", "Volume", "Open", "High", "Low"]]
    df.reset_index()
    df = df.set_index(df.index).resample('D').ffill()
    df = df.reset_index()
    df.columns = df.columns.droplevel(1)
    return df
gatherStockPrices("TSLA")

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Price,Date,Close,Volume,Open,High,Low
0,2015-01-02,14.620667,71466000,14.858000,14.883333,14.217333
1,2015-01-03,14.620667,71466000,14.858000,14.883333,14.217333
2,2015-01-04,14.620667,71466000,14.858000,14.883333,14.217333
3,2015-01-05,14.006000,80527500,14.303333,14.433333,13.810667
4,2015-01-06,14.085333,93928500,14.004000,14.280000,13.614000
...,...,...,...,...,...,...
3647,2024-12-27,431.660004,82666800,449.519989,450.000000,426.500000
3648,2024-12-28,431.660004,82666800,449.519989,450.000000,426.500000
3649,2024-12-29,431.660004,82666800,449.519989,450.000000,426.500000
3650,2024-12-30,417.410004,64941000,419.399994,427.000000,415.750000


In [10]:
totalData = pd.DataFrame(columns = ["Date", "Ticker", "GICS Sector", "Volume", "Close"])
key_set = {"B3 S.A.": ".SA", "Hong Kong Exchanges And Clearing Ltd": ".HK", "London Stock Exchange": ".L", "BSE LTD": ".BO"}
for idx in range(len(allStockTickers)):
    ticker = allStockTickers["Ticker"][idx]
    sector = allStockTickers["GICS Sector"][idx]
    exchange = allStockTickers["Exchange"][idx]
    try:
        if exchange in key_set.keys():
            ticker = ticker + key_set[exchange]
        temp = gatherStockPrices(ticker)
        temp["Ticker"] = [ticker for _ in range(len(temp))]
        temp["GICS Sector"] = [sector for _ in range(len(temp))]
        totalData = pd.concat([totalData, temp], ignore_index=True)
    except:
        print(exchange)
        pass
totalData.to_csv("totalData.csv", index = False)
totalData

[*********************100%***********************]  1 of 1 completed
/var/folders/vb/kfy7llqs5n9c_rfl70nmbxd00000gn/T/ipykernel_23176/1426066928.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  totalData = pd.concat([totalData, temp], ignore_index=True)
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[********

B3 S.A.


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

BMV


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['GAPB']: YFTzMissingError('possibly delisted; no timezone found')


...

Toronto Stock Exchange


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
['ENGH']: YFTzMissingError('possibly delisted; no timezone found')
[*********************100%***********************]  1 of 1 completed


Toronto Stock Exchange


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Date,Ticker,GICS Sector,Volume,Close,Open,High,Low
0,2015-01-02,ABEV3.SA,Consumer Staples,19396500,10.294532,10.371644,10.641538,10.133881
1,2015-01-03,ABEV3.SA,Consumer Staples,19396500,10.294532,10.371644,10.641538,10.133881
2,2015-01-04,ABEV3.SA,Consumer Staples,19396500,10.294532,10.371644,10.641538,10.133881
3,2015-01-05,ABEV3.SA,Consumer Staples,13795900,10.108174,10.223843,10.262400,9.986080
4,2015-01-06,ABEV3.SA,Consumer Staples,18610500,10.493738,10.108175,10.551573,9.998933
...,...,...,...,...,...,...,...,...
1461085,2024-12-27,SU,Energy,3771500,35.259998,35.099998,35.639999,35.029999
1461086,2024-12-28,SU,Energy,3771500,35.259998,35.099998,35.639999,35.029999
1461087,2024-12-29,SU,Energy,3771500,35.259998,35.099998,35.639999,35.029999
1461088,2024-12-30,SU,Energy,3354400,35.290001,35.230000,35.570000,35.009998


In [7]:
combined_features = pd.concat(allFeatures, axis=1)
combined_features.reset_index(inplace=True)
combined_features

,DATE,DAAA
0,2015-01-01,3.148636
1,2015-02-01,3.426000
2,2015-03-01,3.639545
3,2015-04-01,3.522727
4,2015-05-01,3.791429
...,...,...
115,2024-08-01,4.866818
116,2024-09-01,4.460000
117,2024-10-01,4.734783
118,2024-11-01,4.647619


In [12]:
totalData = pd.read_csv("totalData.csv")
totalData['Date'] = pd.to_datetime(totalData['Date']).dt.to_period('M').dt.to_timestamp()
for col in combined_features.columns:
    totalData[col] = None

for idx in range(len(totalData)):
    for pointer in range(len(combined_features)):
        if totalData["Date"].iloc[idx] == combined_features["DATE"].iloc[pointer]:
            totalData.loc[idx, combined_features.columns] = combined_features.loc[pointer, combined_features.columns]
            break

totalData.to_csv("totalDataWithFeatures.csv")
print(totalData)

       Unnamed: 0       Date    Ticker       GICS Sector     Volume  \
0               0 2015-01-01  ABEV3.SA  Consumer Staples  262525100   
1               1 2015-02-01  ABEV3.SA  Consumer Staples  193123300   
2               2 2015-03-01  ABEV3.SA  Consumer Staples  252616400   
3               3 2015-04-01  ABEV3.SA  Consumer Staples  218542400   
4               4 2015-05-01  ABEV3.SA  Consumer Staples  205488100   
...           ...        ...       ...               ...        ...   
48165       48165 2024-08-01        SU            Energy   88896000   
48166       48166 2024-09-01        SU            Energy   96793400   
48167       48167 2024-10-01        SU            Energy   62850300   
48168       48168 2024-11-01        SU            Energy   87835000   
48169       48169 2024-12-01        SU            Energy   94016600   

           Close       Open       High        Low                 DATE  \
0      11.780809  10.888425  11.900682  10.348998  2015-01-01 00:00:00   

# Make sure all tickers have the same length of data

In [ ]:
tickers = totalData["Ticker"].unique()
data_lengths = []
for ticker in tickers:
    data = totalData.loc[totalData["Ticker"] == ticker][["Close"]].values
    data_lengths.append(data.shape[0])
ser = pd.Series(data=data_lengths, index=tickers)

In [50]:
largest_length = ser.value_counts().nlargest(1).index.values
filtered_tickers = ser.loc[ser.values >= largest_length].index.values
filtered_data = totalData.loc[totalData["Ticker"].isin(filtered_tickers)]
filtered_dates = filtered_data.loc[~filtered_data["Date"].isin(["2024-12-31"])]
filtered_dates.to_csv("filteredData.csv", index = False)

/var/folders/vb/kfy7llqs5n9c_rfl70nmbxd00000gn/T/ipykernel_23176/91166940.py:4: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  filtered_dates = filtered_data.loc[~filtered_data["Date"].isin(["2024-12-31"])]


In [51]:
testData = pd.read_csv("filteredData.csv")
tickers = testData["Ticker"].unique()
data_lengths = []
for ticker in tickers:
    data = testData.loc[testData["Ticker"] == ticker][["Close"]].values
    data_lengths.append(data.shape[0])
ser = pd.Series(data=data_lengths, index=tickers)
ser.value_counts()

3651    374
Name: count, dtype: int64

In [125]:
test = [1,2]
test2 = [3,4]
test3 = test + test2
print(test3)

[1, 2, 3, 4]
